In [12]:
import pandas as pd
from astropy import units as u
from astropy.coordinates import SkyCoord

from plato.stars import filter_p1_targets, filter_valid_targets
from plato.utils import get_abspath

In [13]:
LOPS2 = pd.read_csv(get_abspath() + f"data/processed/LOPS2_targets.csv")
LOPN1 = pd.read_csv(get_abspath() + f"data/processed/LOPN1_targets.csv")

LOPN1["Field"] = "LOPN1"
LOPS2["Field"] = "LOPS2"

fields = pd.concat([LOPS2, LOPN1])
fields = filter_valid_targets(fields)
fields = fields[
    [
        "gaiaID_DR3",
        "GLON",
        "GLAT",
        "gaiaV",
        "Population",
        "Field",
        "n_cameras",
        "[Fe/H]",
        "[Fe/H]_apogee",
    ]
]

fields = fields.rename(
    columns={
        "gaiaV": "Magnitude_V",
        "GLON": "l",
        "GLAT": "b",
    }
)

p1_sample = filter_p1_targets(fields)

## FILTER SPECIAL TARGETS

In [3]:
special_targets = (
    p1_sample[p1_sample["Population"] == "Halo"]
    .drop(columns=["Population", "n_cameras"])
    .rename(
        columns={
            "gaiaID_DR3": "Gaia DR3 ID",
            "Magnitude_V": "V Magnitude",
        }
    )
)
print("Number of special targets: ", len(special_targets))

Number of special targets:  0


In [4]:
# create a SkyCoord object with galactic coordinates
galactic_coord = SkyCoord(
    l=special_targets["l"].to_numpy() * u.deg,
    b=special_targets["b"].to_numpy() * u.deg,
    frame="galactic",
)

# convert galactic coordinates to equatorial coordinates
equatorial_coord = galactic_coord.transform_to("icrs")
special_targets["RA"] = equatorial_coord.ra.to_string(
    u.hourangle, sep="hms", pad=True, precision=2
)
special_targets["Dec"] = equatorial_coord.dec.to_string(
    u.deg, sep="dms", pad=True, precision=2
)

special_targets["l"] = special_targets["l"].round(5).astype(str)
special_targets["b"] = special_targets["b"].round(5).astype(str)
special_targets["V Magnitude"] = special_targets["V Magnitude"].round(2).astype(str)

In [5]:
special_targets_table = special_targets[
    ["Gaia DR3 ID", "RA", "Dec", "l", "b", "V Magnitude", "Field"]
]
LOPS2_table = special_targets_table[special_targets["Field"] == "LOPS2"].drop(
    columns=["Field"]
)
LOPN1_table = special_targets_table[special_targets["Field"] == "LOPN1"].drop(
    columns=["Field"]
)

In [6]:
print(LOPS2_table.to_latex(index=False))

\begin{tabular}{rlllll}
\toprule
Gaia DR3 ID & RA & Dec & l & b & V Magnitude \\
\midrule
2892879693284897536 & 06h16m37.01s & -32d27m15.29s & 239.62354 & -20.99928 & 10.89 \\
2911140863435161856 & 05h59m33.51s & -26d16m35.09s & 232.0118 & -22.39392 & 10.91 \\
4662889729476972416 & 04h38m24.22s & -65d24m38.18s & 276.67025 & -38.1506 & 9.82 \\
4676601464106231040 & 04h23m43.36s & -62d17m05.00s & 273.55676 & -40.71957 & 10.52 \\
4761346872572913408 & 05h13m07.19s & -59d38m36.27s & 268.56794 & -35.48969 & 9.39 \\
4768015406298936960 & 05h47m36.18s & -54d09m57.56s & 261.95891 & -30.85913 & 10.57 \\
4772221523607359232 & 05h25m26.69s & -51d58m42.48s & 259.15855 & -34.06364 & 10.98 \\
4797516475799429888 & 05h18m19.97s & -48d52m16.70s & 255.33827 & -35.08485 & 10.63 \\
4818949324679566720 & 05h13m48.21s & -40d54m03.53s & 245.53908 & -35.2076 & 10.73 \\
4864851495597048448 & 04h33m26.34s & -38d18m08.85s & 241.28118 & -42.70393 & 10.96 \\
4874147282294634496 & 04h46m26.38s & -33d09m18.77s & 23

In [7]:
print(LOPN1_table.to_latex(index=False))

\begin{tabular}{rlllll}
\toprule
Gaia DR3 ID & RA & Dec & l & b & V Magnitude \\
\midrule
1340991529725341056 & 17h11m40.84s & 37d49m55.08s & 61.75032 & 35.17462 & 10.59 \\
1342299192648782592 & 17h39m36.23s & 37d10m48.77s & 62.26064 & 29.61662 & 8.36 \\
1400094505966392704 & 16h07m25.17s & 48d41m56.60s & 76.33974 & 46.71623 & 10.93 \\
1423516852416948224 & 16h14m57.15s & 49d46m03.04s & 77.50137 & 45.30106 & 10.98 \\
1622478459328957696 & 15h57m27.57s & 56d40m02.77s & 88.01797 & 45.85473 & 10.68 \\
1644643411153918336 & 15h39m11.90s & 66d48m13.65s & 101.89417 & 42.83402 & 10.31 \\
2026374267595492096 & 19h28m53.81s & 28d22m21.21s & 62.28341 & 5.12122 & 10.29 \\
2051426296414984960 & 19h31m09.21s & 36d09m01.42s & 69.4295 & 8.34166 & 10.2 \\
2077092436860985728 & 19h42m06.28s & 41d41m23.01s & 75.38792 & 9.06408 & 10.62 \\
2083249324019906048 & 20h20m38.47s & 46d26m29.55s & 83.09395 & 5.60762 & 10.81 \\
2104987557947509888 & 18h54m17.01s & 42d59m04.32s & 72.80069 & 17.62626 & 9.93 \\
2107

## SAVE SPECIAL TARGET LIST

In [8]:
LOPS2 = pd.read_csv(get_abspath() + f"data/processed/LOPS2_targets.csv")
LOPN1 = pd.read_csv(get_abspath() + f"data/processed/LOPN1_targets.csv")

LOPN1["Field"] = "LOPN1"
LOPS2["Field"] = "LOPS2"

fields = pd.concat([LOPS2, LOPN1])
fields = filter_valid_targets(fields).rename(columns={"gaiaV": "Magnitude_V"})
p1_sample = filter_p1_targets(fields).rename(columns={"Magnitude_V": "gaiaV"})

special_targets = special_targets = p1_sample[p1_sample["Population"] == "Halo"]
special_targets.to_csv(
    get_abspath() + f"data/processed/special_target_list.csv", index=False
)